In [3]:
!pip install findspark

In [4]:
!pip install pyspark

In [5]:
# Import findspark and initialize.
import findspark
findspark.init()

In [6]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [7]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
# Read the CSV file into a DataFrame
home_sales_revised_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)


In [8]:
# 2. Create a temporary view of the DataFrame.
home_sales_revised_df.createOrReplaceTempView("home_sales")
print(home_sales_revised_df)

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]


In [9]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
result = spark.sql("SELECT date, ROUND(AVG(price), 2) AS average_price FROM home_sales WHERE bedrooms = 4 GROUP BY date ORDER BY date")
result.show()

+----------+-------------+
|      date|average_price|
+----------+-------------+
|2019-01-01|     365088.0|
|2019-01-02|    271644.29|
|2019-01-03|    381218.86|
|2019-01-04|    359586.14|
|2019-01-05|    323683.57|
|2019-01-06|    301091.13|
|2019-01-07|     327474.5|
|2019-01-08|    304567.83|
|2019-01-09|     243619.0|
|2019-01-10|     287844.4|
|2019-01-11|    277574.29|
|2019-01-12|     218507.0|
|2019-01-13|    294603.86|
|2019-01-14|    284434.57|
|2019-01-15|     254288.5|
|2019-01-16|     335106.5|
|2019-01-17|    333439.09|
|2019-01-18|    297542.56|
|2019-01-19|    294484.22|
|2019-01-20|    312395.33|
+----------+-------------+
only showing top 20 rows



In [10]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
result = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS average_price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY date_built ORDER BY date_built")
result.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [11]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

result = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS average_price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 GROUP BY date_built ORDER BY date_built")
result.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [12]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

result = spark.sql("SELECT ROUND(AVG(price), 2) AS average_price FROM home_sales WHERE price >= 350000")
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+
|average_price|
+-------------+
|    473796.26|
+-------------+

--- 0.7543909549713135 seconds ---


In [13]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [14]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [15]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) from home_sales group by view having ROUND(AVG(price),2) >= 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  85|          1056336.74|
|  65|           736679.93|
|  53|            755214.8|
|  78|          1080649.37|
|  81|          1053472.79|
|  76|          1058802.78|
|  91|          1137372.73|
|  93|          1026006.06|
|  52|           733780.26|
|  86|          1070444.25|
|  94|           1033536.2|
|  57|            734340.5|
|  54|           798684.82|
|  96|          1017815.92|
|  92|           970402.55|
|  64|           767036.67|
|  61|           746877.59|
|  88|          1031719.35|
|  72|           780914.67|
|  59|            791453.0|
+----+--------------------+
only showing top 20 rows

--- 4.710524797439575 seconds ---


In [16]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
output_path = "partitioned_data/partitioned_data.parquet"

# Write the DataFrame to Parquet format with partitioning
home_sales_revised_df.write.partitionBy("date_built").parquet(output_path)

In [17]:
# 11. Read the formatted parquet data.
# Specify the path to the formatted Parquet data
parquet_path = "partitioned_data/partitioned_data.parquet"

# Read the Parquet data into a DataFrame
parquet_data = spark.read.parquet(parquet_path)
parquet_data

DataFrame[id: string, date: date, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int, date_built: int]

In [18]:
# 12. Create a temporary table for the parquet data.
parquet_data.createOrReplaceTempView("parquet_table")

In [19]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
spark.sql("SELECT view, ROUND(AVG(price),2) from home_sales group by view having ROUND(AVG(price),2) >= 350000").show()
cached_runtime = time.time() - start_time



start_time = time.time()
spark.sql("SELECT view, ROUND(AVG(price),2) from parquet_table group by view having ROUND(AVG(price),2) >= 350000").show()
parquet_runtime = time.time() - start_time

print("Parquet Runtime: %.2f seconds" % parquet_runtime)
print("Cached Runtime: %.2f seconds" % cached_runtime)

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  85|          1056336.74|
|  65|           736679.93|
|  53|            755214.8|
|  78|          1080649.37|
|  81|          1053472.79|
|  76|          1058802.78|
|  91|          1137372.73|
|  93|          1026006.06|
|  52|           733780.26|
|  86|          1070444.25|
|  94|           1033536.2|
|  57|            734340.5|
|  54|           798684.82|
|  96|          1017815.92|
|  92|           970402.55|
|  64|           767036.67|
|  61|           746877.59|
|  88|          1031719.35|
|  72|           780914.67|
|  59|            791453.0|
+----+--------------------+
only showing top 20 rows

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  85|          1056336.74|
|  65|           736679.93|
|  53|            755214.8|
|  78|          1080649.37|
|  81|          1053472.79|
|  76|          1058802.78|
|  91|          1137372.73|
|  93|          102600

In [20]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [21]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False